In [2]:
import os
import pandas as pd
import numpy as np
import plotnine
from datetime import date
import scipy
import seaborn as sns
from Bio import SeqIO
import matplotlib.pyplot as plt
from upsetplot import from_memberships

In [3]:
## this notebook's purpose is to subset the assemblies by only contigs that are agreed upon
## by some number of assemblers.

In [4]:
## GLOBAL PARAMETERS ##
base_dir = "/vortexfs1/omics/alexander/akrinos/2021-testing-eukrhythmic/eukrhythmic_paper_trials_September21"
eukulele_dir_designer = "EUKulele_designer"
eukulele_dir_CAG = "EUKulele_07-CAG"
eukulele_dir_MAD = "EUKulele_12-MAD"
salmon_dir = os.path.join(base_dir,"08-salmon_mapping")

In [5]:
def count_set(x):
    x = x.dropna()
    y = []
    [y.extend(curr.split("-")) for curr in x]
    try:
        len(sorted(list(set(y))))
    except: 
        print(y,flush=True)
    return len(sorted(list(set(y))))
output_dir="two_way_contigs" #four_way_contigs
num_in_common = 2
os.system("mkdir -p " + output_dir)
## READ IN EUKRHYTHMIC DATA##
jeukebox_dirs = [curr for curr in os.listdir(base_dir) if "jEUKebox" in curr]
all_four_way_contigs = pd.DataFrame()
all_three_way_contigs = pd.DataFrame()
all_two_way_contigs = pd.DataFrame()
all_only_one_way = pd.DataFrame()

def create_set(x):
    x = x.dropna()
    y = []
    [y.extend(curr.split("-")) for curr in x]
    return "-".join(sorted(list(set(y))))

def count_set(x):
    x = x.dropna()
    y = []
    [y.extend(curr.split("-")) for curr in x]
    try:
        len(sorted(list(set(y))))
    except: 
        print(y,flush=True)
    return len(sorted(list(set(y))))

for jeukebox_dir in jeukebox_dirs:
    sourmash_file = os.path.join(base_dir, jeukebox_dir, "sourmash_MMETSP", "all.csv")
    if os.path.isfile(sourmash_file):
        
        if not os.path.isdir(os.path.join(base_dir, jeukebox_dir, 
                             "sourmash_MMETSP")):
            continue
        prot_clust_dir = os.path.join(base_dir, jeukebox_dir,
                                      "eukrhythmic_assembly",
                                      "intermediate-files",
                                      "03-merge", "07-CAG")
        if not os.path.isdir(prot_clust_dir):
            continue
            
        print(jeukebox_dir,"passed prot clust",flush=True)
        cluster_results = pd.DataFrame()
        for sim_file in os.listdir(prot_clust_dir):
            if "tsv" not in sim_file:
                continue
            fasta_file=sim_file.split(".tsv")[0] + ".fasta"
            ## WHAT'S INSIDE THE CAG FINAL CONTIGS??
            cluster_result = pd.read_csv(os.path.join(prot_clust_dir,
                                                      sim_file),sep="\t",
                                        header=None,names=["Representative","Member"])
            comm_num = int(sim_file.split("_merged")[0].split("_")[-1])
            cluster_result["Community"] = int(sim_file.split("_merged")[0].split("_")[-1])
            cluster_result["Name"] = jeukebox_dir
            cluster_result["Assembler_Rep"] = [curr.split("_")[0] for curr in cluster_result.Representative]
            cluster_result["Assembler_Mem"] = [curr.split("_")[0] for curr in cluster_result.Member]
            
            four_way_contigs = cluster_result.groupby(["Community","Name","Representative",
                                                       "Assembler_Rep"]).agg({"Assembler_Mem": [count_set,create_set]}).reset_index()
            four_way_contigs.columns = ['_'.join(col).strip() for col in four_way_contigs.columns.values]
            four_way_contigs_two = four_way_contigs[four_way_contigs["Assembler_Mem_count_set"] >= num_in_common]
            four_way_contigs_three = four_way_contigs[four_way_contigs["Assembler_Mem_count_set"] >= 3]
            four_way_contigs_four = four_way_contigs[four_way_contigs["Assembler_Mem_count_set"] >= 4]
            only_one_way_contigs = four_way_contigs[four_way_contigs["Assembler_Mem_count_set"] == 1]
            all_two_way_contigs= pd.concat([all_two_way_contigs,four_way_contigs_two])
            all_three_way_contigs= pd.concat([all_three_way_contigs,four_way_contigs_three])
            all_four_way_contigs= pd.concat([all_four_way_contigs,four_way_contigs_four])
            all_only_one_way = pd.concat([all_only_one_way,only_one_way_contigs])
            four_way_contigs_list = list(four_way_contigs["Representative_"])
            from Bio import SeqIO

            #with open(os.path.join(prot_clust_dir,fasta_file)) as input_handle, open(
            #    output_dir+"/" + jeukebox_dir + "_" + str(comm_num) + ".fasta", "w"
            #) as output_handle:
            #    sequences = [curr for curr in \
            #     SeqIO.parse(input_handle, "fasta") if curr.id in four_way_contigs_list]
            #    count = SeqIO.write(sequences, output_handle, "fasta")
            

jEUKebox-Trial3-CommB passed prot clust
jEUKebox-Trial2-CommB passed prot clust
jEUKebox-Trial4-100k passed prot clust
jEUKebox-Trial2-100k passed prot clust
jEUKebox-Trial1-CommB passed prot clust
jEUKebox-Trial3-100k passed prot clust
jEUKebox-Trial1-100k passed prot clust
jEUKebox-Trial4-CommB passed prot clust
